In [1]:
# Add rebel component https://github.com/Babelscape/rebel/blob/main/spacy_component.py
import spacy
import crosslingual_coreference
import requests
import re
import hashlib
import pandas as pd
from spacy import Language
from typing import List
from spacy.tokens import Doc, Span
from transformers import pipeline

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\GCM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
def call_wiki_api(item):
    try:
        url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
        data = requests.get(url).json()
        # Return the first id (Could upgrade this in the future)
        return data['search'][0]['id']
    except:
        return 'id-less'

def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})

    return triplets


@Language.factory(
    "rebel",
    requires=["doc.sents"],
    assigns=["doc._.rel"],
    default_config={
        "model_name": "Babelscape/rebel-large",
        "device": 0,
    },
)
class RebelComponent:
    def __init__(
        self,
        nlp,
        name,
        model_name: str,
        device: int,
    ):
        assert model_name is not None, ""
        self.triplet_extractor = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
        self.entity_mapping = {}
        # Register custom extension on the Doc
        if not Doc.has_extension("rel"):
            Doc.set_extension("rel", default={})

    def get_wiki_id(self, item: str):
        mapping = self.entity_mapping.get(item)
        if mapping:
            return mapping
        else:
            res = call_wiki_api(item)
            self.entity_mapping[item] = res
            return res

    
    def _generate_triplets(self, sent: Span) -> List[dict]:
        output_ids = self.triplet_extractor(sent.text, return_tensors=True, return_text=False)[0]["generated_token_ids"]["output_ids"]
        extracted_text = self.triplet_extractor.tokenizer.batch_decode(output_ids[0])
        extracted_triplets = extract_triplets(extracted_text[0])
        return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        for triplet in triplets:

            # Remove self-loops (relationships that start and end at the entity)
            if triplet['head'] == triplet['tail']:
                continue

            # Use regex to search for entities
            head_span = re.search(triplet["head"], doc.text)
            tail_span = re.search(triplet["tail"], doc.text)

            # Skip the relation if both head and tail entities are not present in the text
            # Sometimes the Rebel model hallucinates some entities
            if not head_span or not tail_span:
                continue

            index = hashlib.sha1("".join([triplet['head'], triplet['tail'], triplet['type']]).encode('utf-8')).hexdigest()
            if index not in doc._.rel:
                # Get wiki ids and store results
                doc._.rel[index] = {"relation": triplet["type"], "head_span": {'text': triplet['head'], 'id': self.get_wiki_id(triplet['head'])}, "tail_span": {'text': triplet['tail'], 'id': self.get_wiki_id(triplet['tail'])}}

    def __call__(self, doc: Doc) -> Doc:
        for sent in doc.sents:
            sentence_triplets = self._generate_triplets(sent)
            self.set_annotations(doc, sentence_triplets)
        return doc

In [3]:
DEVICE = 0 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe("xx_coref", 
    config={"chunk_size": 2500, 
            "chunk_overlap": 2, 
            "device": DEVICE}
              )

# Define rel extraction model
rel_ext = spacy.load('en_core_web_sm', disable=['ner', 'lemmatizer', 'attribute_rules', 'tagger'])
rel_ext.add_pipe("rebel", 
                 config={
                     'device':DEVICE,
                     'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
                )

error loading _jsonnet (this is expected on Windows), treating C:\Users\GCM\AppData\Local\Temp\tmpodpmhh5s\config.json as plain json
Some weights of the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large and 

In [4]:
# crosslingual_coreference implementation
def coref_res(text_series):
    coref_text_series = text_series.apply(lambda x : coref(x)._.resolved_text)
    return(coref_text_series)

# # choose minilm for speed/memory and info_xlm for accuracy
# predictor = Predictor(
#     language="en_core_web_sm", device=-1, model_name="minilm"
# )

In [5]:
def link_entities_text(text):
    try:
        ent_rel_lst = list(rel_ext(text)._.rel.values())
    except:
        print("Could not extract relationships for text")
        ent_rel_lst = [{'relation': 'rel_err',
                        'head_span': {'text': 'rel_err', 'id': 'rel_err'},
                        'tail_span': {'text': 'rel_err', 'id': 'rel_err'}}]
        
    entity_df = pd.DataFrame()
    rel_lst = []
    head_text_lst = []
    head_wiki_id_lst = []
    tail_text_lst = []
    tail_wiki_id_lst = []
    for i in range(len(ent_rel_lst)):
        rel_lst.append(ent_rel_lst[i]['relation'])
        head_text_lst.append(ent_rel_lst[i]['head_span']['text'])
        head_wiki_id_lst.append(ent_rel_lst[i]['head_span']['id'])
        tail_text_lst.append(ent_rel_lst[i]['tail_span']['text'])
        tail_wiki_id_lst.append(ent_rel_lst[i]['tail_span']['id'])
    entity_df['head_text'] = head_text_lst
    entity_df['head_wiki_id'] = head_wiki_id_lst
    entity_df['relation'] = rel_lst
    entity_df['tail_text'] = tail_text_lst
    entity_df['tail_wiki_id'] = tail_wiki_id_lst
    return(entity_df)

def link_entities(text_series):
    entity_df_series = text_series.apply(lambda x : link_entities_text(x))
    return(entity_df_series)

In [6]:
data_df = pd.read_csv('E:\\GIT_REPOS\\LAB\\Literature_summary\\Test\\Entity_edgelist\\Input\\WearableBiosensor2017-2022-csv.csv')
# data_df.drop('File', axis=1, inplace=True)
data_df.drop_duplicates(inplace=True)
data_df

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Nam D., Cha J.M., Park K.",36773509400;8844285600;56164630300;,Next-generation wearable biosensors developed ...,2021,Micromachines,12,1,64,1,22,...,NaN,NaN,NaN,English,Micromachines,Article,Final,"All Open Access, Green",Scopus,2-s2.0-85100538965
1,"Roth A.M., Tran N.K., Cocchiaro B., Mitchell A...",35389671400;57202781760;57218686490;5720929184...,Wearable biosensors have the potential to moni...,2021,Drug and Alcohol Dependence,229,NaN,109138,NaN,NaN,...,NaN,DADED,34781097.0,English,Drug Alcohol Depend.,Article,Final,"All Open Access, Green",Scopus,2-s2.0-85119023123
2,"Zhao F., He J., Li X., Bai Y., Ying Y., Ping J.",57189253002;57219272842;57203929081;5630649410...,Smart plant-wearable biosensor for in-situ pes...,2020,Biosensors and Bioelectronics,170,NaN,112636,NaN,NaN,...,NaN,BBIOE,33017772.0,English,Biosens. Bioelectron.,Article,Final,NaN,Scopus,2-s2.0-85091967404
3,"Wei X., Zhu M., Li J., Liu L., Yu J., Li Z., D...",57216639869;57203970066;57222666564;5719631868...,Wearable biosensor for sensitive detection of ...,2021,Nano Energy,85,NaN,106031,NaN,NaN,...,NaN,NaN,NaN,English,Nano Energy,Article,Final,NaN,Scopus,2-s2.0-85103690750
4,"Zhu X., Liu W., Shuang S., Nair M., Li C.-Z.",55537722300;57198618307;7003516023;57203233242...,"Intelligent tattoos, patches, and other wearab...",2017,Medical Biosensors for Point of Care (POC) App...,NaN,NaN,NaN,133,150,...,9780081000786; 9780081000724,NaN,NaN,English,Med. Biosens. for Point of Care (POC) Appl.,Book Chapter,Final,NaN,Scopus,2-s2.0-85016836444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,"Oh J., Lee J.S., Jun J., Kim S.G., Jang J.",56520742700;57201487311;55946625400;5630677010...,Ultrasensitive and Selective Organic FET-type ...,2017,ACS Applied Materials and Interfaces,9,45,NaN,39526,39533,...,NaN,NaN,29067802.0,English,ACS Appl. Mater. Interfaces,Article,Final,NaN,Scopus,2-s2.0-85034792180
1958,"Yu H., Sun J.",16240359300;57219725609;,Sweat detection theory and fluid driven method...,2020,Nami Jishu yu Jingmi Gongcheng/Nanotechnology ...,3,3,NaN,126,140,...,NaN,NaN,NaN,English,Nami Jishu yu Jingmi Gongcheng,Review,Final,"All Open Access, Gold",Scopus,2-s2.0-85094977401
1959,"Wang J., Liu K., Sun Q., Ni X., Ai F., Wang S....",57191042595;57209655285;8881874100;57190001617...,Diaphragm-based optical fiber sensor for pulse...,2019,Journal of Biophotonics,12,10,e201900084,NaN,NaN,...,NaN,NaN,31219245.0,English,J. Biophotonics,Article,Final,NaN,Scopus,2-s2.0-85068425578
1960,"Zhang M., Wang X., Huang Z., Rao W.",57207916058;57219922404;57219921178;55183241300;,Liquid Metal Based Flexible and Implantable Bi...,2020,Biosensors,10,11,170,NaN,NaN,...,NaN,BISSE,33182535.0,English,Biosensors,Review,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85096082336


In [7]:
# import data
# data_df = pd.read_csv('E:\GIT_REPOS\LAB\Literature_summary\TPN\Papers\\scopus.csv')
data_df = data_df[data_df["Abstract"] != '[No abstract available]']
data_df.reset_index(inplace=True, drop=True)
data_df["Abstract"] = data_df["Abstract"].str.replace(r'(', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r')', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r"'", '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r"'", '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r'"', '')
data_df["Abstract"] = data_df["Abstract"].str.replace(r'"', '')
data_df["Abstract"] = data_df["Abstract"].astype(str)

C:\Users\GCM\AppData\Local\Temp\ipykernel_6172\412532311.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_df["Abstract"] = data_df["Abstract"].str.replace(r'(', '')
C:\Users\GCM\AppData\Local\Temp\ipykernel_6172\412532311.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_df["Abstract"] = data_df["Abstract"].str.replace(r')', '')


In [8]:
win_size = 100
start_point = 0 # default to 0
# coref_lst = []
entities_df_lst = []
for i in range(start_point, len(data_df), win_size):
    coref_series = coref_res(text_series=data_df["Abstract"].iloc[i:i+win_size])
    print('coref done', i)
    link_entities_series = link_entities(text_series=coref_series)
    print('entity linking done', i)
    entities_df = pd.concat(link_entities_series.tolist())
    print('df create done', i)
    entities_df_lst.append(entities_df)
    print('df to list done', i, '\n')
all_entities_df = pd.concat(entities_df_lst)
all_entities_df.reset_index(drop=True, inplace=True)
edge_lst_df = all_entities_df.value_counts().reset_index().rename(columns={0: "count"})
edge_lst_df.to_csv('entity_weighted_edgelist_Wearables.csv')

coref done 0
entity linking done 0
df create done 0
df to list done 0 

coref done 100
entity linking done 100
df create done 100
df to list done 100 

coref done 200
entity linking done 200
df create done 200
df to list done 200 

coref done 300
entity linking done 300
df create done 300
df to list done 300 

coref done 400
entity linking done 400
df create done 400
df to list done 400 

coref done 500
Could not extract relationships for text
entity linking done 500
df create done 500
df to list done 500 

coref done 600
entity linking done 600
df create done 600
df to list done 600 

coref done 700
entity linking done 700
df create done 700
df to list done 700 

coref done 800
entity linking done 800
df create done 800
df to list done 800 

coref done 900
entity linking done 900
df create done 900
df to list done 900 

coref done 1000
entity linking done 1000
df create done 1000
df to list done 1000 

coref done 1100
entity linking done 1100
df create done 1100
df to list done 1100 


In [9]:
# coref_series = coref_res(text_series=data_df["Abstract"][:1000])
# link_entities_series = link_entities(text_series=coref_series)
# all_entities_df = pd.concat(link_entities_series.tolist())